<a href="https://colab.research.google.com/github/Pramurta/ETL_Banks_Project/blob/main/ETL_Banks_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
from datetime import datetime
import pandas as pd
import requests
from bs4 import BeautifulSoup
import sqlite3
import numpy as np

In [19]:
def log_progress(message):
    now = datetime.now()
    formatted_date = now.strftime("%Y-%d-%m %H:%M:%S")
    log_line = f"[{formatted_date}] : {message}"
    with open("code_log.txt","a") as f:
        f.write(log_line+"\n")


In [20]:
def extract(url, table_attribs):
    df = pd.DataFrame(columns=table_attribs)

    html_page = requests.get(url).text
    soup = BeautifulSoup(html_page, 'html.parser')
    table = soup.find_all('table')[0]
    tbody = table.find('tbody')
    table_rows = tbody.find_all('tr')
    for i,row in enumerate(table_rows):
        cells = row.find_all("td")
        if len(cells) == 3:
            market_cap_in_usd = cells[2].get_text(strip=True)
            bank_name = cells[1].find_all('a')[-1].get_text(strip=True)
            df.loc[len(df)] = [bank_name, market_cap_in_usd]

    df["MC_USD_Billion"] = df["MC_USD_Billion"].astype(float)

    return df

In [21]:
def transform(df, csv_path):
    exchange_rates_curr_to_usd = pd.read_csv(csv_path)
    df["MC_GBP_Billion"] = np.round(df["MC_USD_Billion"]*exchange_rates_curr_to_usd.loc[exchange_rates_curr_to_usd["Currency"]=="GBP","Rate"].values[0],2)
    df["MC_EUR_Billion"] = np.round(df["MC_USD_Billion"]*exchange_rates_curr_to_usd.loc[exchange_rates_curr_to_usd["Currency"]=="EUR","Rate"].values[0],2)
    df["MC_INR_Billion"] = np.round(df["MC_USD_Billion"]*exchange_rates_curr_to_usd.loc[exchange_rates_curr_to_usd["Currency"]=="INR","Rate"].values[0],2)
    return df

def load_to_csv(df, output_path):
    df.to_csv('Largest_banks_data.csv',index=False)

def load_to_db(df, sql_connection, table_name):
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

def run_query(query_statement, sql_connection):
    df = pd.read_sql_query(query_statement, sql_connection)
    return df

In [22]:
url = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"
banks_data_csv_path = "./Largest_banks_data.csv"
exchange_rates_csv_path = "./exchange_rate.csv"
table_attribs = ["Name", "MC_USD_Billion"]

!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv


log_progress("Data extraction started")
df = extract(url, table_attribs)
log_progress("Data extraction ended, data transformation started")


df = transform(df, exchange_rates_csv_path)

log_progress("Data transformation ended, loading to csv started")

load_to_csv(df, banks_data_csv_path)

log_progress("Loading to csv ended, sql connection starting to establish")


sql_connection = sqlite3.connect('Banks.db')

log_progress("sql connection established, loading to db")


load_to_db(df, sql_connection, "Largest_banks")

log_progress("finished loading to db")


log_progress("running query SELECT * FROM Largest_banks")
query_result = run_query("SELECT * FROM Largest_banks", sql_connection)

log_progress("running query SELECT AVG(MC_GBP_Billion) FROM Largest_banks")
query_result = run_query("SELECT AVG(MC_GBP_Billion) FROM Largest_banks", sql_connection)

log_progress("running query SELECT Name from Largest_banks LIMIT 5")
query_result = run_query("SELECT Name from Largest_banks LIMIT 5", sql_connection)

sql_connection.close()

--2024-07-06 05:27:46--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45 [text/csv]
Saving to: ‘exchange_rate.csv’

exchange_rate.csv   100%[===================>]      45  --.-KB/s    in 0s      

2024-07-06 05:27:46 (12.1 MB/s) - ‘exchange_rate.csv’ saved [45/45]

